In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import requests
import urllib3
import time
import numpy as np

# Get all names from week one 
import scraping
science_people = scraping.get_all_names()

c:\Users\david\Desktop\Computational social science\comsocsci2023\scraping.py:70: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 70 of the file c:\Users\david\Desktop\Computational social science\comsocsci2023\scraping.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(r.content)
c:\Users\david\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ic2s2.mit.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\david\Desktop\Co

In [2]:
#setup
BASE_URL = "https://api.semanticscholar.org/graph/"
VERSION  = "v1/"
RESOURCE = "author/search?query="
complete_url = BASE_URL + VERSION + RESOURCE

In [15]:
# Get all papers from the science people  
authorIds = dict()
for i, name in enumerate(science_people):
    name = name.replace(" ","+").lower()
    
    if i % 100 == 99: # Big sad we can only do 100 request per 5 min - use downtime to save dictionary
        start_time = time.time()
        np.save("autherIds.npy", authorIds)
        time.sleep(60*5+10 + start_time - time.time())
    
    response = requests.get(complete_url + name).json() # Request all papers from the current auther
    
    # Get the ID of the auther and put them into the autherIds dict 
    try: 
        if len(response["data"]):
            authorIds[(response["data"][0]['authorId'])] = list()
    except: 
        print(response)
        print(name)





In [3]:
# Load Ids again 
authorIds = np.load("autherIds.npy", allow_pickle=True).item()
# print(authorIds)

In [7]:
for i, id in enumerate(authorIds.keys()):
    # Make request
    colabs = set()
    full_url = BASE_URL + VERSION + "author/" + id + "?" + "fields=papers.authors"
    
    if i % 100 == 99: # Big sad we can only do 100 request per 5 min - use downtime to save dictionary
        start_time = time.time()
        np.save("autherIds.npy", authorIds)
        time.sleep(60*5+10 + start_time - time.time())
    
    # Send request 
    response = requests.get(full_url).json()
    for paper in response["papers"]:
        for c_id in paper["authors"]:
            colabs.add(c_id["authorId"])
            if "1" == c_id["authorId"]:
                print(c_id)
                time.sleep(5000)
                
    authorIds[id] = colabs - set(id)
    
    
authorIds

{'1381269764': {'1381269764', '1397092505', '71588873'},
 '2083089510': {'152520793',
  '1747572',
  '1825808992',
  '2083089510',
  '2964656',
  '3456882',
  '70473305',
  '8554587'},
 '47960435': {'101263446',
  '10304372',
  '113504443',
  '114369926',
  '114897334',
  '118005652',
  '118821960',
  '121901781',
  '12327330',
  '123505040',
  '12377800',
  '12873699',
  '1384191375',
  '1395560159',
  '1397377225',
  '1398344918',
  '1398589567',
  '1398699741',
  '1398887729',
  '1400763463',
  '1401383320',
  '1401447403',
  '1403690593',
  '1405278123',
  '1405362367',
  '1405594260',
  '1430930197',
  '1435642303',
  '143608343',
  '143715651',
  '143754577',
  '143761079',
  '1437991993',
  '143899108',
  '143909741',
  '143951513',
  '144034058',
  '144102040',
  '144153219',
  '144201362',
  '144205504',
  '144206459',
  '144232865',
  '144391147',
  '144402116',
  '144488967',
  '144511665',
  '144518825',
  '144532701',
  '144621386',
  '144638344',
  '144810960',
  '1448820

In [15]:
# Collect all authors into one set
all_ids_set = set()
for (key, item) in authorIds.items():
    item.add(key)
    all_ids_set.update(item)


False


In [16]:
# Names and aliases for each author
# Make into dict 
all_ids = {}
for i, id in enumerate(all_ids_set):
    # Make a request for the authors aliases, name, and papers 
    full_url = BASE_URL + VERSION + "author/" + str(id)
    params = "fields=aliases,name,papers.title,papers.abstract,papers.year,papers.externalIds,papers.s2FieldsOfStudy,papers.citationCount"
    
    if i % 100 == 99: # Big sad we can only do 100 request per 5 min - use downtime to save dictionary
        start_time = time.time()
        np.save("all_ids.npy", all_ids)
        time.sleep(60*5+10 + start_time - time.time())
    
    # Make request 
    req = requests.get(full_url,params=params)  
    all_ids[id] = {"name": req.json()["name"], 
                   "aliases": req.json()["aliases"],
                   "papers": req.json()["papers"]}
    

    

In [8]:
# Load Ids again 
all_ids = np.load("all_ids.npy", allow_pickle=True).item()
print(all_ids["1740681"].keys())

dict_keys(['name', 'aliases', 'papers'])


In [14]:
print(all_ids["1740681"]["papers"][0]["s2FieldsOfStudy"])

[{'category': 'Computer Science', 'source': 's2-fos-model'}]


In [30]:
# Create people dataframe 
zero_data = np.zeros((len(all_ids.keys()), 5))
author_df = pd.DataFrame(zero_data, columns=["id", "name", "aliases", "citationCount", "field"])

for i, id in enumerate(all_ids.keys()):
    # ID
    author_df["id"][i] = id 
    information = all_ids[str(id)]
    # Name
    author_df["name"][i] = information["name"]
    # Aliases
    author_df["aliases"][i] = information["aliases"] 
    # citation count 
    citation_count = 0
    for paper in information["papers"]: 
        citation_count += paper["citationCount"]
    author_df["citationCount"][i] = citation_count
    # field
    potential_fields = {}
    for paper in information["papers"]: 
        for fields in paper["s2FieldsOfStudy"]:
            field = fields["category"]
            try:
                potential_fields[field] += 1
            except: 
                potential_fields[field] = 1
    if potential_fields == {}: 
        pass
    else: 
        author_df["field"][i] = max(potential_fields, key=potential_fields.get)

author_df

{'Computer Science': 20, 'Biology': 1, 'Medicine': 17, 'Mathematics': 4, 'Physics': 6, 'Materials Science': 2, 'Psychology': 1, 'Engineering': 2}
{'Medicine': 64, 'Biology': 30, 'Engineering': 1}
{'Geology': 5, 'Environmental Science': 21, 'Psychology': 2, 'Engineering': 2, 'Agricultural And Food Sciences': 2, 'Chemistry': 1, 'Materials Science': 1}
{'Engineering': 90, 'Physics': 5, 'Environmental Science': 9, 'Agricultural And Food Sciences': 1, 'Computer Science': 2, 'Business': 2, 'Materials Science': 1, 'Geography': 1, 'Art': 1, 'Chemistry': 1}
{'Psychology': 100, 'History': 3, 'Art': 2, 'Business': 5, 'Education': 22, 'Sociology': 1, 'Economics': 1, 'Medicine': 1, 'Political Science': 1, 'Philosophy': 1}
{'Linguistics': 1, 'Computer Science': 37, 'Education': 2, 'Business': 3, 'Economics': 2, 'Mathematics': 1}
{'Economics': 1, 'Linguistics': 26, 'Computer Science': 20, 'Psychology': 1, 'Biology': 1, 'Education': 1}
{'Psychology': 2, 'Medicine': 2, 'Computer Science': 2, 'Business'

C:\Users\david\AppData\Local\Temp/ipykernel_63124/3289348295.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  author_df["aliases"][i] = information["aliases"]
c:\Users\david\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\david\AppData\Local\Temp/ipykernel_63124/3289348295.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

ValueError: max() arg is an empty sequence

In [29]:
import requests
BASE_URL = "https://api.semanticscholar.org/graph/"
VERSION = "v1/"
RESOURCE = "author/batch"


params = {'query':"I",
          "ids":["1741101", "1780531", "48323507"]}

my_url = BASE_URL + VERSION + RESOURCE
r = requests.get(my_url, params=params)
r.json()

{'error': 'Author with id batch not found'}

In [19]:
hej = {}
for i in range(3):
    try:
        hej["hej"] += 1 
    except:
        hej["hej"] = 1

hej.get

<function dict.get(key, default=None, /)>